# Analyse et visualisation de réseaux:

## Import des scripts et description de la base:

Dans ce script, nous allons voir comment calculer les metriques de base décrivant la structure des réseaux, et comment visualiser un réseau.

Pour cela, nous utilserons les réseaux d'édition de deux pages du wikipédia anglophone: 

- Zapata rail (Oiseau Cubain)
- Oil Spill (Marées noires)

On commence par importer les bases pour Zapata rail, qui se trouvent dans le sous dossier *Data*:

In [ ]:
zapata_rail_edit_edgelist <- read.csv2(file = "Data/Zapata_rail_edit_edgelist.csv")
zapata_rail_edit_attributes <- read.csv2(file = "Data/Zapata_rail_edit_attributes.csv")

In [ ]:
# Options pour les graphes: on modifie la taille de l'affichage

options(repr.plot.width = 15, repr.plot.height = 10)

Comment sont organisées les bases que l'on vient d'importer?

In [ ]:
head(zapata_rail_edit_edgelist)

La base "edgelist" décrit une suite d'action qui ont lieux sur la page. Ici, le réseau d'édition est organisé de la manière suivante: 

Les contributeurs peuvent effectuer trois types d'action sur une page, pour chaque action, le type est décrit dans la variable InteractionType: 

- ADDED: Ajout de texte à la page, la cible est toujours le noeud "page".
- DELETED: Suppression de texte, la cible est l'auteur du texte original.
- RESTORED: Annule une suppression ou modification de texte. La cible est l'auteur original du texte qui est restauré.

On peut voir leur distribution à l'aide de la fonction table(base$variable):


In [ ]:
table(zapata_rail_edit_edgelist$InteractionType)

Les contributeurs (ActiveUser) effectuent ainsi toujours une action envers un autre contributeur, ou la page. La cible de l'action est décrite dans la variable TargetAuthor.

Dans le but de simplifier les bases, j'ai regroupé les actions similaires (ex: l'utilisateur x reverse trois fois du texte écrit par y). La variable InteractionCount décrit le nombre total d'actions similaires décrites par la ligne.

La variable Wordcount décrit le nombre total de mots concernés par la ligne (ajouté à la page pour un ADD, supprimés pour un DELETE, etc.)

In [ ]:
# Pour avoir deux graphes côte à côte:

par(mfrow=c(1,2))

boxplot(zapata_rail_edit_edgelist$WordCount, xlab = "WordCount",
        main = "Nombre de mots par action")

boxplot(zapata_rail_edit_edgelist$InteractionCount, xlab = "InteractionCout",
        main = "Nombre d'interaction similaires")

par(mfrow=c(1,1))

In [ ]:
head(zapata_rail_edit_attributes)

La base d'attributs décrit les charactéristiques des utilisateurs listés dans le réseau d'édition. Là ou la base *edgelist* nous montre une ligne par action, ici, la base nous montre une ligne par contributeur unique qui a modifié à un moment la page Zapata Rail.

La base nous donne le statut du contributeur ($Status). Celui ci peut être:

- Anon: Contributeur anonyme, non inscrit sur wikipédia. Il est identifié par l'adresse IP utilisée lors de la modification.
- Page: La page apparait dans la liste des attributs pour des raisons techniques. Elle n'a évidemment pas d'autres caractéristiques que d'être "la page".
- Admin: Administrateurs wikipédia: utilisateurs élus disposant de droits d'édition et de modération élargis.
- Bot: Robots pour la mise en forme, la correction, et l'édition automatique.
- Users: Contributeurs inscrits, identifiés par leur pseudos.

On observe la distribution des status des auteurs de la page à l'aide de la fonction table:

In [ ]:
table(zapata_rail_edit_attributes$Status)

La variable TotalWordCount nous donne le nombre total de mots contribués par l'utilisateur à la page (ADD uniquement donc).

In [ ]:
boxplot(zapata_rail_edit_attributes$TotalWordCount, xlab = "InteractionCout",
        main = "Nombre d'interaction similaires")

## Visualisation du graphe:

On peut maintenant commencer à regarder le réseau. Le package R que l'on utilise pour ce cours est igraph. Il faut en premier lieu le charger avec la commande library(package), en prenant soin de ne charger que les colonnes décrivant une action (lien), ici, les colonnes 1 et 2:

In [ ]:
library(igraph)

Il nous faut ensuite charger la base qui décrit les liens (notre base 'edgelist')  dans un objet graphe:

In [ ]:
zapata_rail_edit_graphe <- graph.edgelist(as.matrix(zapata_rail_edit_edgelist[,2:3]))

On peut ensuite regarder ce qu'il y  a dans l'objet graphe que l'on vient de créer:

In [ ]:
summary(zapata_rail_edit_graphe)

**Lecture de la sortie:** C'est un DN (*Directed Network* = Réseau dirigé), qui comporte 110 noeuds, et 371 liens.  Il ne comporte pour l'instant pas d'attributs (pour l'instant), à part *name* qui décrit le nom des noeuds.

On peut réaliser une première représentation du graphe avec la commande plot:

In [ ]:
plot(zapata_rail_edit_graphe)

Problème, c'est assez peu lisible...

On peut ajuster le graphe en ajoutant des arguments à la fonction plot. 

On peut enlever les labels:

In [ ]:
plot(zapata_rail_edit_graphe,
     vertex.label= NA)

Réduire la taille des arrêtes (edge) et des noeuds (vertex) :

In [ ]:
plot(zapata_rail_edit_graphe,
     edge.arrow.size=.3,
     vertex.size = 8,
     vertex.label= NA)

On peut aussi visualiser les attributs des noeuds et des liens directement sur le graphe, à l'aide de conventions graphiques.

Il faut d'abord les charger depuis leurs bases respectives dans l'objet graphe (edgelist pour les liens et attributes pour les noeuds): Ci dessous, V appelle l'objet noeuds et E l'objet liens: 

In [ ]:
# On charge le status des contributeurs:

V(zapata_rail_edit_graphe)$Status <- zapata_rail_edit_attributes$Status

# Le nombre total de mots contribués à la page

V(zapata_rail_edit_graphe)$TotalWordCount <- zapata_rail_edit_attributes$TotalWordCount

# Le nombre d'"Actions" contenues dans un lien (nombre de fois que ce lien exact a lieu)

E(zapata_rail_edit_graphe)$IterationCount <- zapata_rail_edit_edgelist$InteractionCount

# Le type d'action: ADD, DELETE ou REVERSE?

E(zapata_rail_edit_graphe)$InteractionType <- zapata_rail_edit_edgelist$InteractionType

# On isole la page:

V(zapata_rail_edit_graphe)$Page <- ifelse(zapata_rail_edit_attributes$Users == "Page", 'square', 'circle')

On peut ensuite ajouter ces attributs au graphe, sous la forme que l'on veut:
Ici, je choisis de faire varier la taille des liens en fonction du nombre d'"Actions" contenues dans un lien, la taille des noeuds en fonction du nombre total de mots qu'ils ont ajoutés à la page, et la couleur des noeuds en fonction de leur statut.

In [ ]:
plot(zapata_rail_edit_graphe,
     edge.arrow.size = .3,
     edge.width = E(zapata_rail_edit_graphe)$IterationCount,
     edge.color = as.factor(E(zapata_rail_edit_graphe)$InteractionType),
     vertex.size = V(zapata_rail_edit_graphe)$TotalWordCount,
     vertex.color = as.factor(V(zapata_rail_edit_graphe)$Status),
     vertex.label = NA)

Problème, notre taille de noeuds a une amplitude beaucoup trop importante. Plusieurs solutions existent, on peut prendre le log, on peut centrer-réduire. Ici, je prends le log * 2, +4 pour que la taille minimale des points soit de 4:

In [ ]:
plot(zapata_rail_edit_graphe,
     edge.arrow.size = .3,
     edge.width = E(zapata_rail_edit_graphe)$IterationCount,
     edge.color = as.factor(E(zapata_rail_edit_graphe)$InteractionType),
     vertex.size = log(V(zapata_rail_edit_graphe)$TotalWordCount + 4) * 2,
     vertex.color = as.factor(V(zapata_rail_edit_graphe)$Status),
     vertex.label = NA,
     vertex.shape = V(zapata_rail_edit_graphe)$Page)

Problème: On ne sait pas trop quelles couleurs correspondent à quel statut ou a quelle interaction...
On peut définir des vecteurs de couleur à la main, en choisissant les correspondances:

In [ ]:
E(zapata_rail_edit_graphe)$Color_edge[E(zapata_rail_edit_graphe)$InteractionType == "ADDED"] <- "#3892e0"
E(zapata_rail_edit_graphe)$Color_edge[E(zapata_rail_edit_graphe)$InteractionType == "DELETED"] <- "#da4d45"
E(zapata_rail_edit_graphe)$Color_edge[E(zapata_rail_edit_graphe)$InteractionType == "RESTORED"] <- "#fbd25d"

V(zapata_rail_edit_graphe)$Color_nodes[V(zapata_rail_edit_graphe)$Status == "Anon"] <- "white"
V(zapata_rail_edit_graphe)$Color_nodes[V(zapata_rail_edit_graphe)$Status == "User"] <- "#f37329"
V(zapata_rail_edit_graphe)$Color_nodes[V(zapata_rail_edit_graphe)$Status == "Bot"] <- "#93d844"
V(zapata_rail_edit_graphe)$Color_nodes[V(zapata_rail_edit_graphe)$Status == "Admin"] <- "#8a4ebf"
V(zapata_rail_edit_graphe)$Color_nodes[V(zapata_rail_edit_graphe)$Status == "Page"]<- "#333333"

Et les appliquer à notre graphe précédant:

In [ ]:
plot(zapata_rail_edit_graphe,
     edge.arrow.size = .3,
     edge.width = E(zapata_rail_edit_graphe)$IterationCount,
     edge.color = E(zapata_rail_edit_graphe)$Color_edge,
     vertex.size = log(V(zapata_rail_edit_graphe)$TotalWordCount + 4) * 2,
     vertex.color = V(zapata_rail_edit_graphe)$Color_nodes,
     vertex.label = NA,
     vertex.shape = V(zapata_rail_edit_graphe)$Page)

On peut rajouter une légende qui décrit les marques l'on a ajouté à la visualisation:

In [ ]:
plot(zapata_rail_edit_graphe,
     edge.arrow.size = .3,
     edge.width = E(zapata_rail_edit_graphe)$IterationCount,
     edge.color = E(zapata_rail_edit_graphe)$Color_edge,
     vertex.size = log(V(zapata_rail_edit_graphe)$TotalWordCount + 4) * 2,
     vertex.color = V(zapata_rail_edit_graphe)$Color_nodes,
     vertex.label = NA,
     vertex.shape = V(zapata_rail_edit_graphe)$Page)


legend(x="topleft", c("Ajout","Suppression","Reverse","admin","anonyme","inscrit", "bot",
                      "page","WordCount -", "WordCount +"),
       pch=c(24,24,24,21,21,21,21,22,21,21),
       col="#777777", 
       pt.bg= c("#3892e0","#da4d45","#fbd25d","#8a4ebf","white","#f37329",
                "#93d844", "#333333", "white", "white"),
       pt.cex=c(2,2,2,2,2,2,2,2,1,4),
       cex=.8,
       bty="n",
       ncol=1)

### Exercice 1:

Dans le dossier *Data* se trouve deux autre bases décrivant le réseau d'édition de la page "Oil Spill" (Oil_spill_edit_edgelist.csv et Oil_spill_edit_edgelist.csv). Visualisez le réseau d'édition. Quelle différences observe t-on entre nos deux réseaux? Quelles hypothèses peut-on émettre pour expliquer ces différences?

In [ ]:
# Chargez les bases:

# Créez l'objet graphe:

# Chargez les attributs:

# Attribuez les couleurs correctes:

# Et lancez la visualisation:

## Calcul de metriques structurelles:

## La description du réseau dans son ensemble:

A partir de l'objet graphe, on peut calculer un certain nombre de metriques descriptives de la structure globale du réseau. On a déjà vu comment réccupérer le nombre de noeuds et de liens, mais on peu aussi regarder:

La densité (le nombre de liens existant par rapport au nombre de liens possibles):

In [ ]:
graph.density(zapata_rail_edit_graphe)

Le nombre de composantes (combien de sous réseaux séparés composent notre réseau)

In [ ]:
clusters(zapata_rail_edit_graphe)

Le taux de réciprocité des liens (pour les graphes dirigés):

In [ ]:
reciprocity(zapata_rail_edit_graphe)

La distance moyenne entre les noeuds:

In [ ]:
average.path.length(zapata_rail_edit_graphe)

Le diamètre (distance entre les deux noeuds les plus éloignés du réseau)

In [ ]:
diameter(zapata_rail_edit_graphe)

L'eccentricité: la distance depuis un noeud de départ vers le noeud le plus loin dans le réseau)

In [ ]:
eccentricity(zapata_rail_edit_graphe)

Le rayon: (la plus petite distance à laquelle un noeud peut se trouver de tout les autres)

In [ ]:
radius(zapata_rail_edit_graphe)

La transitivité globale: nombre de triades transitives par rapport au nombre de triades totales

In [ ]:
transitivity(zapata_rail_edit_graphe)

La transitivité locale et moyenne: combien de noeuds connectés à un noeud sont aussi connectés entre-eux?

In [ ]:
trans_locale <- transitivity(zapata_rail_edit_graphe, type = "local")
mean(trans_locale, na.rm = T)

Le nombre et la taille des cliques: (noeuds tous en interaction les uns avec les autres)

In [ ]:
cl <- cliques(zapata_rail_edit_graphe, min=3)

length(cl)

largest_cliques(zapata_rail_edit_graphe) 

clique_num(zapata_rail_edit_graphe)

## Nouveaux attributs des noeuds: variables de réseau et centralité:

En plus des métriques décrivant la structure globale du réseau, on peut calculer les différentes mesures de centralité (voir slides) pour chaque contributeur. On dispose ainsi d'un attribut supplémentaire, décrivant le rôle de chaque contribteur dans la structure du résau.

On peut ainsi calculer les demi-degrés intérieurs et extérieurs:

In [ ]:
V(zapata_rail_edit_graphe)$DegIn <- degree(zapata_rail_edit_graphe, mode="in")
V(zapata_rail_edit_graphe)$DegOut <- degree(zapata_rail_edit_graphe, mode="out") 

Les décrire:

In [ ]:
mean(V(zapata_rail_edit_graphe)$DegIn)
sd(V(zapata_rail_edit_graphe)$DegIn)

mean(V(zapata_rail_edit_graphe)$DegOut)
sd(V(zapata_rail_edit_graphe)$DegOut)

Et visualiser leur distribution:

In [ ]:
par(mfrow=c(1,2))

hist(V(zapata_rail_edit_graphe)$DegIn, col = "lightblue",
     main = "Degré entrant des contributeurs de la page Zapata rail",
     xlab = "InDegree")

hist(V(zapata_rail_edit_graphe)$DegOut, col = "lightblue",
     main = "Degré sortant des contributeurs de la page Zapata rail",
     xlab = "OutDegree")

par(mfrow=c(1,1))

On peut calculer la centralisation du réseau:

In [ ]:
# Indegree

centr_degree(zapata_rail_edit_graphe, mode = "in", normalized = T)$centralization

# OutDegree 

centr_degree(zapata_rail_edit_graphe, mode = "out", normalized = T)$centralization

La centralité d'intermédiarité pour tous les noeuds:

In [ ]:
V(zapata_rail_edit_graphe)$Bet <- betweenness(zapata_rail_edit_graphe, directed=TRUE, normalized = FALSE)

mean(V(zapata_rail_edit_graphe)$Bet)
sd(V(zapata_rail_edit_graphe)$Bet)

Pour finir par la centralité de vecteurs propres. Pour cela on a besoin de construire un graphe non orienté:

In [ ]:
zapata_rail_edit_graphe_no <- graph.edgelist(as.matrix(zapata_rail_edit_edgelist[,2:3]), directed = F)

Puis on calcule la centralité d'intermédiarité:

In [ ]:
eigen_centrality(zapata_rail_edit_graphe_no, directed = FALSE)$value

hist(eigen_centrality(zapata_rail_edit_graphe_no, directed = FALSE)$vector, col = "lightblue",
     main = "Eigen Centrality", xlab = "Eigen value")

### Exercice 2:

Calculez les metriques pour le reseau. Que peut-on conclure des différences entre les deux réseaux? Celà nous aide t-il à réponde aux hypothèses posées lors de la phase de visualisation?

In [ ]:
# Pas besoin de recréer l'objet graphe

# Indice: Les metriques qui nous intéressent ici sont principalement:
# la densité, la centralisation et les demi-degrés intèrieurs et extérieurs

# Vous pouvez aussi comparer la distribution des status entre les deux réseaux.